### Day 11: Plutonian Pebbles

Link: https://adventofcode.com/2024/day/11#part2

Given `n` is now 75, the previous `O(2ⁿ)` approach no longer works in reasonable time without optimizations, so we need some insights.

The first thing we can notice is that each number from the initial list can be processed independently from the others. We can use this to extract the processing step into a method that receives the number as a parameter and returns a result list with either one or two numbers, following the problem's description.

The most important thing to notice is that if we start with the number `0`, after a certain number of steps, the numbers start to cycle. No matter how many more steps you process, the same 54 numbers will be revisited. Since it's a cycle, this also works if we start with any of those numbers. We can use this to create a lookup that calculates the final result in at most `54 * n`. The lookup would look something like this: `0` creates one occurrence of `1`. `1` creates one occurrence of `2024`. `2024` creates one occurrence of `20` and one occurrence of `24`, and so on. This allows us to multiply the occurrences of the current numbers by the occurrences of the numbers they generate, saving us from adding them one by one.

For numbers outside that cycle, like `999`, we can still process them with the `O(2ⁿ)` method. As we keep processing, if we find any number that is in the cycle, we use the fast method to calculate its final result with the number of steps left and avoid processing it further. This periodic pruning prevents the list of numbers from growing exponentially. We then add the length of the final list of numbers to the result to get the answer.

In [ ]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
import typing as t
from collections import Counter
from functools import cache


def make_new_numbers(number: int) -> list[int]:
    if number == 0:
        return [1]

    number_str = str(number)

    if len(number_str) % 2 == 0:
        mid_idx = len(number_str) // 2
        return [int(number_str[:mid_idx]), int(number_str[mid_idx:])]

    return [number * 2024]


numbers_lookup: dict[int, t.Counter[int]] = {}
numbers_to_process = [0]


while numbers_to_process:
    new_numbers_to_process: list[int] = []

    for number in numbers_to_process:
        if number in numbers_lookup:
            continue

        new_numbers = make_new_numbers(number)
        numbers_lookup[number] = Counter(new_numbers)
        new_numbers_to_process.extend(new_numbers)

    numbers_to_process = new_numbers_to_process


@cache
def calculate_final_size(number: int, steps: int) -> int:
    number_counter = {number: 1}

    for _ in range(steps):
        new_number_counter: dict[int, int] = {}

        for number, occurrences in number_counter.items():
            for new_number, new_number_occurrences in numbers_lookup[number].items():
                new_number_counter.setdefault(new_number, 0)
                new_number_counter[new_number] += new_number_occurrences * occurrences

        number_counter = new_number_counter

    return sum(number_counter.values())


result = 0
numbers = list(map(int, lines[0].strip().split()))
step = 0
max_steps = 75


while numbers and step < max_steps:
    new_numbers: list[int] = []

    for number in numbers:
        if number in numbers_lookup:
            result += calculate_final_size(number, max_steps - step)
        else:
            new_numbers.extend(make_new_numbers(number))

    numbers = new_numbers
    step += 1


result += len(numbers)
print(result)